In [ ]:
from datetime import date
import copy
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from scipy.interpolate import LinearNDInterpolator as Li
from scipy.interpolate import NearestNDInterpolator as Ni
from scipy.spatial import Delaunay as Dl

import zonal_som as ZS


In [ ]:
today  = date.today()
yymmdd = today.strftime("%Y%m%d")

case="cesm2"

if case=="cesm1":
    ifi="/glade/p/cesmdata/cseg/inputdata/ocn/docn7/SOM/pop_frc.b.e11.B1850LENS.f09_g16.pi_control.002.20190923.nc"
    ofi="pop_frc.b.e11.B1850LENS.f09_g16.pi_control.002.Idealized_NOANNUAL."+yymmdd+".nc"
if case=="cesm2":
    ifi="/glade/p/cesmdata/cseg/inputdata/ocn/docn7/SOM/pop_frc.b.e20.B1850.f09_g17.pi_control.all.297.20180523.nc"
    ofi="pop_frc.b.e20.B1850.f09_g17.pi_control.all.297.Idealized_NOANNUAL."+yymmdd+".nc"
if case=="cesm3":
    ifi="/glade/p/cesmdata/cseg/inputdata/ocn/docn7/SOM/mom_frc_b.cesm3_cam058_mom_e.B1850WscMOM.ne30_L58_t061.camdev_cice5.026c_50-99_c20221111.nc"
    ofi="/mom_frc_b.cesm3_cam058_mom_e.B1850WscMOM.ne30_L58_t061.camdev_cice5.026c_50-99.Idealized_NOANNUAL."+yymmdd+".nc"

In [ ]:
sm=xr.open_dataset(ifi)
zsm=copy.deepcopy( sm )

xc=sm['xc'].values
yc=sm['yc'].values

flds0=list( sm.variables )

flds=['hblt','qdp']

print("Fields in input SOM ",flds0)
print("will only replace ",flds)
print("will output to ",ofi)

arj=zsm.area.values
msk=zsm.mask.values
yc=zsm.yc.values
arx=np.where( (msk == 0) , 0., arj ) 


for ifld in flds:
    print("  .., doing ",ifld)
    if (ifld=='hblt'):
        fcq=zsm[ ifld ]
    
        for imo in np.arange(12):
            fcq[ imo ,:,:] = np.where( (msk == 0) ,  fcq[ imo ,:,:] , 45.3 ) 

    if (ifld=='qdp'):
        fcq=zsm[ ifld ]
    
        for imo in np.arange(12):
            fcq[ imo ,:,:] = np.where( ( (msk != 0)&(yc>-30)&(yc<30.) ), 30., 0. ) 
            fcq[ imo ,:,:] = np.where( ( (msk != 0)&(yc<=-30) ), -30., fcq[ imo ,:,:] ) 
            fcq[ imo ,:,:] = np.where( ( (msk != 0)&(yc>=30.) ), -30., fcq[ imo ,:,:] ) 
 
            hav=np.sum( fcq[ imo ,:,:].values.flatten() * arx.flatten() )
            har=np.sum( arx.flatten() )
            bias = hav/har 
            fcq[ imo ,:,:] = np.where( (msk != 0), fcq[ imo ,:,:] - bias , fcq[ imo ,:,:]) 


    zsm[ ifld ] = fcq


zsm.to_netcdf( ofi  )

In [ ]:
####

## zsm=xr.open_dataset(ofi)


In [ ]:
###
arj=zsm.area.values
msk=zsm.mask.values

arx=np.where( (msk == 0) , 0., arj ) 

hav=np.sum( zsm.qdp[0,:,:].values.flatten() * arx.flatten() )
har=np.sum( arx.flatten() )

bias = hav/har 
print( f"Starting bias {bias}")



In [ ]:
plt.plot( zsm.hblt.values.flatten(),'.')

In [ ]:
#####
hfix = zsm.hblt.values

## The next command sets horig to zero wherever it is 'not finite.
## The '~' is a 'bitwise NOT operator' --- thanks chatGPT
hfix[~np.isfinite(hfix)] = 0

In [ ]:
###

#plt.tricontourf( sm.xc.values.flatten(),sm.yc.values.flatten(),horig[0,:,:].flatten() , levels=np.linspace(0,100))
plt.tricontourf( zsm.xc.values.flatten(),zsm.yc.values.flatten(),hfix[0,:,:].flatten() , levels=np.linspace(0,50))

#plt.tricontourf( zsm.xc.values.flatten(),zsm.yc.values.flatten(),arx.flatten() , levels=21 )
#plt.tricontourf( zsm.xc.values.flatten(),zsm.yc.values.flatten(),zsm.area.values.flatten() , levels=21 )

#plt.tricontourf( zsm.xc.values.flatten(),zsm.yc.values.flatten(),zsm.mask.values.flatten() , levels=[-1,0,1,2,3,4,5,6,7,8,9,10,11] )
#plt.tricontourf( zsm.xc.values.flatten(),zsm.yc.values.flatten(),zsm.qdp[0,:,:].values.flatten() , levels=np.linspace(-50,50)  )

plt.colorbar()
#plt.ylim(-100,1000)

In [ ]:
plt.plot( zsm.yc.values.flatten(),zsm.qdp.mean(axis=0).values.flatten() , '.')
plt.ylim(-35,35)